# Sim-launcher
This script show how to launch sims using Python

### 1. Package imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import os
import psycopg2
from psycopg2.extras import execute_values
import random

### 2. Environment Variables

In [2]:
# Load the environment variables
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
# Print this to see if the env variables are read now
os.getenv("COMPOSE_PROJECT_NAME")

'evidss'

### 3. Database connection (writer)

In [3]:
# Generic function to test the connection to the database
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(    
            host=os.getenv("MAIN_HOST"),
            database=os.getenv("MAIN_DB"),
            user=os.getenv("DBWRITE_USER"),
            password=os.getenv("DBWRITE_PWD"), 
            port = os.getenv("MAIN_PORT")
        )
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [4]:
# Make the test database connection
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-11), 64-bit',)
Database connection closed.


In [37]:
conn = psycopg2.connect(    
    host=os.getenv("MAIN_HOST"),
    database=os.getenv("MAIN_DB"),
    user=os.getenv("DBWRITE_USER"),
    password=os.getenv("DBWRITE_PWD"), 
    port = os.getenv("MAIN_PORT")
)

# create a cursor
cur = conn.cursor()

### 4. Database queries

#### 4.1 Base-case analysis

In [6]:
sql_set = 'INSERT INTO analysis_sets (description) VALUES (%s);'
set_data = 'first set as test'

In [7]:
sql_analysis = 'INSERT INTO analysis_record (user_id, status, include_tesla) VALUES (%s, %s, %s);'
analysis_data = (os.getenv("AUTH0_USERID"), 'inserted', 'FALSE')

In [8]:
sql_user = 'INSERT INTO user_details (user_id, user_name, email_id) VALUES (%s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET last_submit_date = NOW();'
user_data = (os.getenv("AUTH0_USERID"), os.getenv("AUTH0_USERNAME"), os.getenv("AUTH0_USERNAME"))

In [9]:
sql_params = 'INSERT INTO analysis_params (param_id, param_value) VALUES %s'
params_data = [(1, '123' ),(2, '70' ),(14, '10' ),(3, '80' ),(4, '100' ),(9, '40' ),(10, '50' ),(11, '25' ),(12, '23' ),(13, '20' ), (15, '1' ), ( 16, '10' ), (17, '80' ), (18, '0' ), (19, '60' ), (20, '20' ), (21, '200' )]

##### Launch an analysis

In [14]:
# cur.mogrify(sql_set, (set_data,))

In [11]:
# All these should be executed together as a transaction
################### the following will launch a sim 
cur.execute(sql_set, (set_data, ))
cur.execute(sql_analysis,  analysis_data)
cur.execute(sql_user,  user_data)
execute_values(cur, sql_params, params_data)
conn.commit()

Launch a set of analysis requests

In [25]:
################### The following will launch 5 sims with varying seed 
#######################################################################
create_new_set = True # a boolean to encode whether to create a new set for this analysis request or add this to the previous one 
number_of_sims = 5 # launch five sims

for i in range(0, number_of_sims): 
    seed = random.randint(1, 1000)
    if(create_new_set):
        set_data = 'Testing the effect of varying seed with everything else constant'
        cur.execute(sql_set, (set_data, ))
    cur.execute(sql_analysis,  analysis_data)
    cur.execute(sql_user,  user_data)
    # change the seed 
    params_data.pop(0) # remove the current list element for parameter 'global_seed' (param_id = 1)
    params_data.insert(0, (1, str(seed)))
    execute_values(cur, sql_params, params_data)
    create_new_set = False # since the next 4 simulations belong to the same set
    conn.commit()

#### 4.2 Adding new chargers

In [24]:
# The order of columns in the csv is important
new_evse_scenario = pd.read_csv('new_evse_scenario.csv')
new_evse_scenario

,latitude,longitude,dcfc_plug_count,dcfc_power,level2_plug_count,level2_power,dcfc_fixed_charging_price,dcfc_var_charging_price_unit,dcfc_var_charging_price,dcfc_fixed_parking_price,dcfc_var_parking_price_unit,dcfc_var_parking_price,level2_fixed_charging_price,level2_var_charging_price_unit,level2_var_charging_price,level2_fixed_parking_price,level2_var_parking_price_unit,level2_var_parking_price,connector_code
0,48.370848,-118.768157,1,50,1,10,0.5,min,0.5,0.5,min,0.5,0.5,min,0.5,0.5,min,0.5,3


In [25]:
new_evse_scenario.dtypes

latitude                           float64
 longitude                         float64
 dcfc_plug_count                     int64
 dcfc_power                          int64
 level2_plug_count                   int64
 level2_power                        int64
 dcfc_fixed_charging_price         float64
 dcfc_var_charging_price_unit       object
 dcfc_var_charging_price           float64
 dcfc_fixed_parking_price          float64
 dcfc_var_parking_price_unit        object
 dcfc_var_parking_price            float64
 level2_fixed_charging_price       float64
 level2_var_charging_price_unit     object
 level2_var_charging_price         float64
 level2_fixed_parking_price        float64
 level2_var_parking_price_unit      object
 level2_var_parking_price          float64
 connector_code                      int64
dtype: object

In [34]:
new_evse_data = [tuple(row) for row in new_evse_scenario.itertuples(index=False)] 

In [35]:
sql_new_evse = """INSERT INTO new_evses (latitude, longitude,
                dcfc_plug_count, dcfc_power, level2_plug_count, level2_power,
                dcfc_fixed_charging_price, dcfc_var_charging_price_unit,
                dcfc_var_charging_price, dcfc_fixed_parking_price, dcfc_var_parking_price_unit,
                dcfc_var_parking_price, level2_fixed_charging_price, level2_var_charging_price_unit,
                level2_var_charging_price, level2_fixed_parking_price, level2_var_parking_price_unit,
                level2_var_parking_price, connector_code) VALUES %s"""

In [38]:
# All these should be executed together as a transaction
set_data = 'testing a new evse scenario using code'
cur.execute(sql_set, (set_data, ))
cur.execute(sql_analysis,  analysis_data)
cur.execute(sql_user,  user_data)
execute_values(cur, sql_params, params_data)
if (len(new_evse_scenario.index) > 0):
    execute_values(cur, sql_new_evse, new_evse_data)
# use execute values with new evses by creating an array of tuples 
conn.commit()